In [1]:
import os
import sys
from getpass import getpass
sys.path.append("../")

In [2]:
# Makes it easier to iterate
%load_ext autoreload
%autoreload 2

In [3]:
# We'll be using OpenAI to evaluate locally so we have to set the API key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key:  ········


In [4]:
# ================================================================================================
# This represents our example LLM application structure
# ================================================================================================

import time
from lynxius.rag.types import ContextChunk
from lynxius.evals.context_precision import ContextPrecision
from lynxius.evals.bert_score import BertScore
from lynxius.tracing.observe import lynxius_observe, lynxius_finalize

@lynxius_observe
def pipeline(query):
    vector = embed(query)
    contexts = retrieve(vector, topk=5)

    # Evaluate the contexts
    lynxius_contexts = [ContextChunk(document=c["document"], relevance=c["score"]) for c in contexts]
    context_precision = ContextPrecision(label="live_rag_lookup", tags=["live", "traced", "rag"])
    context_precision.add_trace(query=query, reference="-", context=lynxius_contexts)
    context_precision.evaluate_local()

    full_response = LLM(
        template="answer this: {query}, given this: {contexts}",
        data={"query": query, "contexts": contexts},
    )
    
    response = LLM(
        template="summarize this: {text}",
        data={"text": full_response["text"]},
    )

    # Evaluate the summarization
    bert_score = BertScore("live_summarization", level="word", presence_threshold=0.65, tags=["live", "traced", "summarization"])
    bert_score.add_trace(reference=full_response["text"], output=response["text"])
    bert_score.evaluate_local()
    
    return response


@lynxius_observe
def embed(query):
    vector = [0.4, 0.3, 0.8, 0.9, 0.1, 0.2]
    time.sleep(1.5)
    return vector


@lynxius_observe
def retrieve(vector, topk):
    contexts = rag_db(vector, topk)
    reranked = rerank(contexts, topk=2)
    time.sleep(0.5)
    return reranked


@lynxius_observe
def rag_db(query, topk):
    contexts = [
        {"document": "text 1 here", "score": 0.7},
        {"document": "text 2 here", "score": 0.8},
        {"document": "text 3 here", "score": 0.1},
        {"document": "text 4 here", "score": 0.2},
        {"document": "text 5 here", "score": 0.9},
    ]
    time.sleep(0.4)
    return contexts


@lynxius_observe
def rerank(contexts, topk):
    time.sleep(1)
    return contexts[:topk]


@lynxius_observe
def LLM(template, data, model="gpt-4o"):
    time.sleep(2)
    return {
        "text": "Answer produced by the LLM",
        "num_tokens": 1234,
        "model_version": "gpt-4o_v123",
    }

In [5]:
from lynxius.client import LynxiusClient

# Run your pipeline with tracing
pipeline("User query here")

# Get the trace
trace = lynxius_finalize()

# And store it in a dedicated Lynxius project!
client_testing = LynxiusClient(api_key="PU7Mf8iDMVcH2ElMaabChQP6zkLqb2cTbrlfnIagGAuHhWyj", run_local=True)
client_testing.store_traces([trace])

['cd1bcfdb-93fb-4e30-b61a-63315183a9ab',
 '0074ea3b-1783-49f6-9cde-a254654debe8']